In [4]:
import numpy as np
from io import StringIO
import pandas as pd
import pyomo.environ as pyo

# Data from Problem 12.3 from the book Model Building in Mathematical Programming (H. Paul Williams)
# Two entries in the top row changed
data = """10 6 8 4 11 9 3
0.5 0.7 – – 0.3 0.2 0.5
0.1 0.2 – 0.3 – 0.6 –
0.2 – 0.8 – – – 0.6
0.05 0.03 – 0.07 0.1 – 0.08
 – – 0.01 – 0.05 – 0.05"""

# Column names for the big Data Frame
Columns = ["Profit", "Grinding", "V_drilling", "H_drilling", "Boring", "Planing"]

Production = pd.read_csv(StringIO(data), 
                         header = None, 
                         sep = "\s+", 
                         na_values=["–"])
Production.fillna(0, inplace = True)
Production = Production.transpose()
Production.columns = ["Profit", "Grinding", "V_drilling", "H_drilling", "Boring", "Planing"]
Production.reindex(list(range(Production.shape[0])))
Production.index.name = "Product"

Profit = Production["Profit"]
# A little modification to the profit data to make the products more "competitive" against each other
Profit[0] = 7
Profit[4] = 8

# Remove the Profit column from the data frame because we have it elsewhere
Production.drop("Profit", 
                  axis=1,
                  inplace=True)

# Numbers of machines of different types
Nb_machines = pd.Series([4, 2, 3, 1, 1], 
                        index = ["Grinding", "V_drilling", "H_drilling", "Boring", "Planing"])
# total hours per month = 24 working days times 8 hours
Hours = 24 * 8

In [6]:
# Solving the nominal production problem

m = pyo.ConcreteModel('Production planning')

products = list(Profit.index)
#Variables = how much of each product we make
m.p = pyo.Var(products, 
              within=pyo.NonNegativeReals)

m.constraints = pyo.ConstraintList()

# for each resource we have the availability constraint
for resource in Production.columns:
    m.constraints.add(pyo.quicksum(m.p[product] * Production.loc[product, resource] for product in products) <= 
                      Hours * Nb_machines[resource])

# Maximize the total profit
m.obj = pyo.Objective(expr = pyo.quicksum(m.p[product] * Profit.loc[product] for product in products),
                     sense = pyo.maximize)

solver = pyo.SolverFactory('gurobi')
solver.solve(m)

nominal_plan = pd.Series({i: m.p[i]() for i in products}, name = "Nominal")
display(nominal_plan)

0       0.000000
1     117.791411
2     720.000000
3       0.000000
4    1884.662577
5     600.736196
6       0.000000
Name: Nominal, dtype: float64

In [14]:
# We shall now solve the robust problem with each (product, machine) time deviating
# by at most max_perturbation * 100%, 
# and per machine the  at most unc_budget products in total deviate by their max
max_perturbation = 0.05
unc_budget = 1

Production_Perturbation = Production.applymap(lambda x : x * max_perturbation)

m = pyo.ConcreteModel('Production planning')

products = list(Profit.index)
resources = list(Production.columns)

# Old variables
m.p = pyo.Var(products, 
              within=pyo.NonNegativeReals)
# Variable which will act as a proxy on s >= abs(duration - nominal duration)
m.s = pyo.Var(products, 
              resources,
              within=pyo.NonNegativeReals)
# Dual variable related to the budget constraint in the uncertainty set
m.lam = pyo.Var(resources,
              within=pyo.NonNegativeReals)

m.constraints = pyo.ConstraintList()

# for each resource we have the availability constraint
for resource in resources:
    m.constraints.add(unc_budget * m.lam[resource] + 
                      pyo.quicksum(m.p[product] * Production.loc[product, resource] for product in products) +
                      pyo.quicksum(m.s[product, resource] for product in products)
                      <= Hours * Nb_machines[resource])
    
# constraints that enforce the role s plays
for resource in resources:
    for product in products:
        m.constraints.add(m.s[product, resource] >= 
                          m.p[product] * Production_Perturbation.loc[product, resource] - m.lam[resource])
        
m.obj = pyo.Objective(expr = pyo.quicksum(m.p[product] * Profit.loc[product] for product in products),
                     sense = pyo.maximize)

solver = pyo.SolverFactory('gurobi')
solver.solve(m)

# Extract the solution
robust_plan = pd.Series({i: m.p[i]() for i in products}, name = "Robust")
production_plans = pd.concat([nominal_plan, robust_plan], axis = 1)
production_plans.index.name = "Product"
display(production_plans)

,Nominal,Robust
Product,,
0,0.000000,0.000000
1,117.791411,128.260238
2,720.000000,685.714286
3,0.000000,0.000000
4,1884.662577,1791.925646
5,600.736196,568.806274
6,0.000000,0.000000


In [15]:
production_plans_profits = production_plans.apply(lambda x: sum([x[i] * Profit[i] for i in x.index]), 
                                                  axis = 0)
display(production_plans_profits)

Nominal    26950.674847
Robust     25709.937347
dtype: float64